In [2]:
from ibm_quantum_widgets import CircuitComposer

from sympy.physics.quantum import pauli
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi
import numpy as np
from qiskit.quantum_info.operators import Operator, Pauli

In [3]:
qreg_q = QuantumRegister(1, 'q')
circuit = QuantumCircuit(qreg_q)



clreg = ClassicalRegister(1)

sx = pauli.SigmaX()
sy = pauli.SigmaY()
sz = pauli.SigmaZ()

$|\psi\rangle = e^{-i\phi/2}\cos(\theta/2)|0\rangle + e^{i\phi/2}\sin(\theta/2)|1\rangle$

In [4]:
def to_spherical(state):
    r0 = np.abs(state[0])
    ϕ0 = np.angle(state[0])
    r1 = np.abs(state[1])
    ϕ1 = np.angle(state[1])
    r = np.sqrt(r0 ** 2 + r1 ** 2)
    θ = 2 * np.arccos(r0 / r)
    ϕ = ϕ1 - ϕ0
    return [r, θ, ϕ]

def to_cartesian(polar):
    r = polar[0]
    θ = polar[1]
    ϕ = polar[2]
    x = r * np.sin(θ) * np.cos(ϕ)
    y = r * np.sin(θ) * np.sin(ϕ)
    z = r * np.cos(θ)
    return [x, y, z]

In [40]:
def rn_su2_1(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(n1/2.0)*np.exp(1j/2.0*(n2+n3)), np.sin(n1/2)*np.exp(-(1j/2.0)+(n2-n3))],
    [-np.sin(n1/2.0)*np.exp(1j/2.0*(n2-n3)), np.cos(n1/2)*np.exp(-1j/2.0*(n2+n3))]
    #ToDo: find unitary SU(2) matrix
    ])

print(rn_su2_1(pi,1,1,1))

Operator([[ 0.47415988+0.73846026j,  0.42073549-0.22984885j],
          [-0.47942554+0.j        ,  0.47415988-0.73846026j]],
         input_dims=(2,), output_dims=(2,))


In [41]:
def rn_su2_1a(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(n1/2.0)*np.exp(1j/2.0*(n2+n3)), np.sin(n1/2)*np.exp(-(1j/2.0)+(n2-n3))],
    [-np.sin(n1/2.0)*np.exp(1j/2.0*(n2-n3)), np.cos(n1/2)*np.exp(-1j/2.0*(n2+n3))]
    #ToDo: find unitary SU(2) matrix
    ])

print(rn_su2_1a(pi,1,1,1))

Operator([[ 0.47415988+0.73846026j,  0.42073549-0.22984885j],
          [-0.47942554+0.j        ,  0.47415988-0.73846026j]],
         input_dims=(2,), output_dims=(2,))


In [42]:
def rn_su2_2(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.exp(1j+(n1+n2)/2)+np.cos(0.5+n3) , 1j+np.exp(1j*(n1-n2)/2)*np.sin(0.5+n3)],
    [1j+np.exp(1j*(n2-n1)/2)*np.sin(0.5*n3) , np.exp(-1j*(n1+n2)/2)*np.cos(0.5*n3)]
    #ToDo: find unitary SU(2) matrix
    ])

print(rn_su2_2(pi,1,1,1))

Operator([[1.53943114+2.28735529j, 0.99749499+1.j        ],
          [0.47942554+1.j        , 0.47415988-0.73846026j]],
         input_dims=(2,), output_dims=(2,))


In [ ]:
states = []
points = []

alpha = 1/np.sqrt(2)
beta = 1/np.sqrt(2)
s = np.array([alpha,beta])
state = Statevector(s)
states.append(state)

Ψ = [complex(alpha, 0), complex(beta, 0)]
polar = to_spherical(Ψ)
pnt = to_cartesian(polar)
points.append(pnt)

rotated = state

for i in range(0,10):

    #Apply unitary gate specified by obj to qubits.
    rotated = circuit.unitary(rn_su2_2(pi,1,1,1),1)


    pnt = [(sx*rotated), (sy*rotated), (sz*rotated)]

    #ToDo: patch function
    r = np.sqrt(pnt[0]**2+pnt[1]**2+pnt[2]**2)

    states.append(rotated)
    points.append(pnt)

#ToDo: Add Points and Vectors to qubit
circuit.add_bits(points)